In [1]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import pandas as pd
import pint
import sys
from typing import List

In [2]:
sys.path.append(str(Path.cwd().parent))

In [3]:
from scr.utils import safe_parse_quantity
from scr.comsol_module.comsol_classes import COMSOL_VTU
from scr.comsol_module.entropy import calculate_S_therm

In [4]:
ROOT = Path().cwd().parent
PARAMETER_SPACE = "03"
ROOT

PosixPath('/Users/thomassimader/Documents/NIRB')

In [5]:
mapped_root = ROOT / "data" / PARAMETER_SPACE / "TrainingMapped"
assert mapped_root.exists()
mapped_folders : List[Path]= [folder for folder in mapped_root.iterdir() if not folder.is_file()]

In [6]:
mapped_dic = {}
for folder in mapped_folders:
    npy_file = folder / "Exports" / "Training_temperatures.npy"
    vti_file = [file for file in folder.rglob("*.vt*")][0]
    mapped_dic[folder.stem] = {"npy": npy_file, "vti" : vti_file, "entropy_num" : [], "entropy_gen" : []}

In [7]:
joblib_file = ROOT / "data" / PARAMETER_SPACE / "Training_Original" / "Training_temperatures.joblib"
assert joblib_file.exists()
vtu_file = sorted([file for file in joblib_file.parent.rglob("*.vtu")])
original_dic = {}
original_dic["original"] = {"joblib": joblib_file, "vtu" : vtu_file, "entropy_num" : [], "entropy_gen" : []}
original_dic["original"]["n_cells"] = []

## Calculate entropy number

In [8]:
param_folder = ROOT / "data" / PARAMETER_SPACE / "Exports"
assert param_folder.exists()
param_files = sorted([path for path in param_folder.rglob("Training*.csv")])
print(len(param_files))

100


### Mapped Meshes for different sizes

In [10]:
for idx, (key, values) in enumerate(mapped_dic.items()):
    param_df = pd.read_csv(param_files[idx], index_col = 0)
    ureg = pint.UnitRegistry()
    param_df['quantity_pint'] = param_df[param_df.columns[-1]].apply(lambda x : safe_parse_quantity(x, ureg))
    lambda_therm = (1 - param_df.loc['host_phi', "quantity_pint"]).magnitude * param_df.loc['host_lambda', "quantity_pint"].magnitude + \
                param_df.loc['host_phi', "quantity_pint"].magnitude * (4.2)
    T0 = 0.5 * (param_df.loc["T_h", "quantity_pint"] + param_df.loc["T_c", "quantity_pint"])
    delta_T = (param_df.loc['T_h', "quantity_pint"]  - param_df.loc["T_c", "quantity_pint"])
    comsol_data = COMSOL_VTU(values["vti"])
    mapped_dic[key]["n_cells"] = comsol_data.mesh.n_cells
    temperatures = np.load(values["npy"])
    for temperature in temperatures:
        comsol_data.mesh.point_data["temp_field"] = temperature[0]
        cell_mesh = comsol_data.mesh.point_data_to_cell_data()
        cell_mesh.cell_data.keys()
        temp_grad = cell_mesh.compute_derivative("temp_field").cell_data["gradient"]
        s0 = calculate_S_therm(lambda_therm,
                        T0.magnitude,
                        temp_grad)
        s0_total = np.sum(s0 * comsol_data.mesh.compute_cell_sizes()["Volume"])
        L = comsol_data.mesh.bounds.z_max - comsol_data.mesh.bounds.z_min
        s0_characteristic = (lambda_therm * delta_T.magnitude**2) / (L**2 * T0.magnitude**2)
        entropy_number = s0_total / s0_characteristic / comsol_data.mesh.volume 
        mapped_dic[key]["entropy_gen"].append(s0_total)
        mapped_dic[key]["entropy_num"].append(entropy_number)


### Original COMSOL data

In [11]:
for idx, file in enumerate(original_dic["original"]["vtu"]):
    param_df = pd.read_csv(param_files[idx], index_col = 0)
    ureg = pint.UnitRegistry()
    param_df['quantity_pint'] = param_df[param_df.columns[-1]].apply(lambda x : safe_parse_quantity(x, ureg))
    lambda_therm = (1 - param_df.loc['host_phi', "quantity_pint"]).magnitude * param_df.loc['host_lambda', "quantity_pint"].magnitude + \
                param_df.loc['host_phi', "quantity_pint"].magnitude * (4.2)
    T0 = 0.5 * (param_df.loc["T_h", "quantity_pint"] + (20 + 273.15) * ureg.kelvin)
    delta_T = (param_df.loc['T_h', "quantity_pint"] - (20 + 273.15) * ureg.kelvin)
    comsol_data = COMSOL_VTU(file)
    temp_field = comsol_data.format_field("Temperature", -1)
    cell_mesh = comsol_data.mesh.point_data_to_cell_data()
    cell_mesh.cell_data.keys()
    temp_grad = cell_mesh.compute_derivative(temp_field).cell_data["gradient"]
    s0 = calculate_S_therm(lambda_therm,
                    T0.magnitude,
                    temp_grad)
    s0_total = np.sum(s0 * comsol_data.mesh.compute_cell_sizes()["Volume"])
    L = comsol_data.mesh.bounds.z_max - comsol_data.mesh.bounds.z_min
    s0_characteristic = (lambda_therm * delta_T.magnitude**2) / (L**2 * T0.magnitude**2)
    entropy_number = s0_total / s0_characteristic / comsol_data.mesh.volume 
    original_dic["original"]["entropy_gen"].append(s0_total)
    original_dic["original"]["entropy_num"].append(entropy_number)
    original_dic["original"]["n_cells"].append(comsol_data.mesh.n_cells)

## Plot

In [12]:
N_SNAPS = len(original_dic["original"]["vtu"])
colors = px.colors.sample_colorscale("jet", [n/(N_SNAPS) for n in range(N_SNAPS)])
print(N_SNAPS)

100


In [27]:
fig = go.Figure()

squarred_errors = np.zeros((N_SNAPS, len(mapped_folders) + 1)) #! Only if original is also included
for idx_snap in range(N_SNAPS):
    entropies = []
    cell_sizes = []
    mapped_meshes = []


    for key, value in mapped_dic.items():
        entropies.append(value["entropy_num"][idx_snap])
        cell_sizes.append(value["n_cells"])
        mapped_meshes.append(key)
        
    sorted_indices = np.argsort(np.array(cell_sizes))
    entropies = np.array(entropies)[sorted_indices]
    cell_sizes = np.array(cell_sizes)[sorted_indices]
    mapped_meshes_sorted = [mapped_meshes[i] for i in sorted_indices]
    
    original_n_cells = original_dic["original"]["n_cells"][idx_snap]
    original_entropy_num = original_dic["original"]["entropy_num"][idx_snap]
    
    idx_org = np.searchsorted(cell_sizes, original_n_cells)
    cell_sizes = np.insert(cell_sizes, idx_org, original_n_cells)
    entropies = np.insert(entropies, idx_org, original_entropy_num)
    mapped_meshes_sorted.insert(idx_org, "Original Comsol Mesh")
    

    rel_error = (entropies - original_entropy_num) / original_entropy_num
    squarred_errors[idx_snap, :] = (entropies - original_entropy_num)**2
    
    mapped_meshes_sorted = np.array(mapped_meshes_sorted, dtype=object)
    idx_snap_array = np.ones_like(rel_error) * idx_snap
    # Stack into 2D array: shape (n_points, 2)
    custom_data = np.column_stack((mapped_meshes_sorted, rel_error * 100, idx_snap_array))
    
    fig.add_trace(go.Scatter(x=cell_sizes,
                             y=entropies,
                                mode='lines + markers',
                                name=f"{idx_snap:03d}",
                                opacity=0.4,#
                                customdata=custom_data,
                                hovertemplate=
                                    'Cells: %{x}<br>'
                                    'Entropy number: %{y}<br>'
                                    'Snap Idx: %{customdata[2]:03d}<br>'
                                    'Mapped: %{customdata[0]}<br>'
                                    'Rel Error: %{customdata[1]:.3f}%<extra></extra>',
                                line=dict(color=colors[idx_snap])
                ))
    
    # fig.add_trace(go.Scatter(x=[original_n_cells],
    #                          y=[original_entropy_num],
    #                          mode='markers',
    #                          name=f"{idx_snap:03d} - org",
    #                         #  opacity=0.4,
    #                          line=dict(color= "black"),  #colors[idx_snap] ) )
    #             ))
    
fig.update_layout(
    showlegend=False,        # Hide the legend
    title=f"Entropy generation number vs n_cells - Training Snapshots - Parameter Space {PARAMETER_SPACE}",       # Optional: add a title
    xaxis_title="n_cells [-]",    # Optional: label for x-axis
    yaxis_title="Entropy number [-]",     # Optional: label for y-axis
    xaxis=dict(
        type='log',                               # Set x-axis to logarithmic scale
        tickformat='.1e',                         # Format x-axis ticks to scientific notation with 2 decimal places
    ),
    yaxis=dict(
        tickformat='.2f',                         # Format y-axis ticks to 2 decimal places (optional)
    )
)

fig.write_image(mapped_root / "Training_Diff_EntropyGeneration.png")
fig.write_html(mapped_root / "Training_Diff_EntropyGeneration.html")
fig.show()

### Errors

In [28]:
mses = np.sum(squarred_errors, axis=0)/N_SNAPS
meses_dic = {mesh: mse for (mse, mesh) in zip(mses, mapped_meshes_sorted) }
for key, val in meses_dic.items():
    print(f"{key} - MSE : {val:>10.1e}")

s200_200_200_b0_4000_0_5000_-4000_-0 - MSE :    1.6e-02
Original Comsol Mesh - MSE :    0.0e+00
s100_100_100_b0_4000_0_5000_-4000_-0 - MSE :    1.8e-04
s50_50_50_b0_4000_0_5000_-4000_-0 - MSE :    8.5e-03
s25_25_25_b0_4000_0_5000_-4000_-0 - MSE :    2.0e-02
